In [1]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
from rsna_retro.train3d import *

Loading imports


In [2]:
torch.cuda.set_device(3)

In [3]:
dls = get_3d_dls_aug(Meta.df_comb, sz=128, bs=32, grps=Meta.grps_stg1)

In [4]:
x,y = dls.one_batch()
x.shape, y.shape

(torch.Size([32, 60, 3, 128, 128]), torch.Size([32, 60, 6]))

## Model

In [5]:
class NeuralNet(nn.Module):
    def __init__(self, n_classes=6, embed_size=1024, LSTM_UNITS=1024, DO = 0.3):
        super(NeuralNet, self).__init__()
        
#         self.embedding_dropout = SpatialDropout(0.0) #DO)
        self.flat = nn.Sequential(AdaptiveConcatPool2d(), Flatten())
        self.hook = ReshapeBodyHook(self.flat)
        
        self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True)

        self.linear1 = nn.Linear(LSTM_UNITS*2, LSTM_UNITS*2)
        self.linear2 = nn.Linear(LSTM_UNITS*2, LSTM_UNITS*2)

        self.linear = nn.Linear(LSTM_UNITS*2, n_classes)

    def forward(self, x):
        h_embedding = self.flat(x)
        h_embadd = torch.cat((h_embedding, h_embedding), -1)
        
        h_lstm1, _ = self.lstm1(h_embedding)
        h_lstm2, _ = self.lstm2(h_lstm1)
        
        h_conc_linear1  = F.relu(self.linear1(h_lstm1))
        h_conc_linear2  = F.relu(self.linear2(h_lstm2))
        
#         print([x.shape for x in [h_lstm1, h_lstm2, h_conc_linear1, h_conc_linear2, h_embadd]])
        
        hidden = h_lstm1 + h_lstm2 + h_conc_linear1 + h_conc_linear2 + h_embadd

        output = self.linear(hidden)
        
        return output

In [6]:
m = NeuralNet()

In [7]:
config=dict(custom_head=m)
learn = get_learner(dls, xresnet18, get_loss(), config=config)

In [8]:
hook = ReshapeBodyHook(learn.model[0])
learn.add_cb(DePadLoss())

In [9]:
learn.load(f'runs/baseline_stg1_xresnet18-3', strict=False)

In [12]:
name = 'train3d_baseline_lstm_2nd'

In [10]:
# learn.summary()

## Training

In [12]:
# learn.lr_find()

In [13]:
learn.freeze()
learn.fit_one_cycle(1, 1e-2)
learn.unfreeze()

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.078723,0.086629,0.972733,0.946604,08:25


In [14]:
learn.save(f'runs/{name}-freeze')

## 384 - no_freeze

In [15]:
learn.load(f'runs/{name}-freeze')

In [16]:
learn.dls = get_3d_dls_aug(Meta.df_comb, path=path_jpg, sz=384, bs=4, grps=Meta.grps_stg1)

In [17]:
do_fit(learn, 4, 1e-3)
learn.save(f'runs/{name}-2-no256')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.082066,0.083042,0.972893,0.946489,1:07:17
1,0.066043,0.074193,0.975681,0.954026,1:07:03
2,0.056587,0.066686,0.978379,0.957680,1:06:55
3,0.047035,0.062194,0.979723,0.961500,1:06:53


## Test

In [13]:
learn.load(f'runs/{name}-2-no256', strict=True)
sub_fn = f'subm/{name}'

In [14]:
dls = get_3d_dls_aug(Meta.df_tst, sz=384, path=path_tst_jpg, bs=8, test=True)

In [15]:
learn.dls = dls

In [16]:
# learn = get_learner(dls, learn.model)
# hook.deregister()
# hook = ReshapeBodyHook(learn.model[0])
# learn.add_cb(DePadLoss())

In [17]:
preds,targs = learn.get_preds()

In [18]:
preds.shape, preds.min(), preds.max()

(torch.Size([121232, 6]), tensor(1.2397e-13), tensor(0.9998))

In [19]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [20]:
api.competition_submit(f'{sub_fn}.csv', name, 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:02<00:00, 10.3MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [29]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14325387,
 'totalBytes': 27277209,
 'date': '2020-01-29T19:15:10.047Z',
 'description': 'train3d_baseline_lstm_2nd',
 'errorDescription': None,
 'fileName': 'train3d_baseline_lstm_2nd.csv',
 'publicScore': '0.85209',
 'privateScore': '0.05333',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14325387/14325387.raw'}